# Image Offsets: Process sequence

This notebook transforms the procedures developed in `Offsets_1.ipynb` into callable functions. 

These functions are used in a loop to process an entire sequence of images. Results are later examined in plots.

In [ ]:
%pylab notebook
%matplotlib notebook

import os, glob

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from astropy.table import Table, vstack
from astropy.stats import SigmaClip

import photutils
from photutils import Background2D, ModeEstimatorBackground, DAOStarFinder

import rawpy

In [ ]:
# parameters for background subtraction and star finding
bkg_sigma = 3.0
bkg_cell_footprint = (100, 100)
bkg_filter = (5, 5)
dao_fwhm = 3.0
dao_threshold = 10.

# operators
sigma_clip = SigmaClip(sigma=bkg_sigma)
bkg_estimator = ModeEstimatorBackground()

In [ ]:
# 1st test image - this will be the reference image against with subsequent images
# will have their offsets computed. We need to read it here to get the camera color 
# array specification as well.
fname = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/DSC03779.ARW'
raw = rawpy.imread(fname)
imarray = raw.raw_image_visible.astype(float)

In [ ]:
# masks that isolate the RGB pixels - these are camera-dependent and work with all images
colors_array = raw.raw_colors_visible

red_mask = np.where(colors_array == 0, 1, 0)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = green_mask_1 | green_mask_2

blue_mask = np.where(colors_array == 2, 1, 0)

In [ ]:
# normalization factors for the RGB arrays
red_norm = 1.321875  # smooth background
green_norm = 1.
blue_norm = 1.27695312

In [ ]:
def get_offsets(sources, sources_prev):
    sources_out.add_column(np.nan, name='xoffset')
    sources_out.add_column(np.nan, name='yoffset')
    sources_out.add_column(0, name='ref_row')
    sources_out.add_column(0, name='prev_row')

    # loop over rows in previous table
    for row_index_prev in range(len(sources_prev)):
        # index in reference table
        ref_row = sources_prev[row_index_prev]['ref_row']

        # if previous table does not contain a pointer to the reference 
        # table, ignore.
        if ref_row == 0:
            continue

        # get position in previous table
        x_prev = sources_prev[row_index_prev]['xcentroid']
        y_prev = sources_prev[row_index_prev]['ycentroid']

        # loop over rows in current table
        for row_index in range(len(sources3)):
            x = sources[row_index]['xcentroid']
            y = sources[row_index]['ycentroid']

            # offsets in relation to previous table - these are the ones to check for proximity
            x_off_previous = x - x_prev
            y_off_previous = y - y_prev

            # check for proximity, and store relevant info if found
            if abs(x_off_previous) <= 1.5 and abs(y_off_previous) <= 1.5:

                # offsets in relation to reference table
                sources[row_index]['xoffset'] = x - sources[ref_row]['xcentroid']
                sources[row_index]['yoffset'] = y - sources[ref_row]['ycentroid']

                # store pointers to rows in reference and previous tables
                sources[row_index]['ref_row'] = ref_row
                sources[row_index]['prev_row'] = row_index_prev

                break # if there is another star that matches the criterion, just ignore it

In [ ]:
def find_stars(image_name, sources_table_previous=None):
    
    raw = rawpy.imread(image_name)
    imarray = raw.raw_image_visible.astype(float)
    
    # normalize
    raw_norm_1 = imarray * (red_mask * red_norm)
    raw_norm_2 = raw_norm_1 + imarray * (green_mask * 1.0)
    raw_norm = raw_norm_2 + imarray * (blue_mask * blue_norm)

    # handle saturated pixels
    raw_norm = np.where(imarray > 16380, imarray, raw_norm)

    # compute and subtract background
    bkg = Background2D(raw_norm, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    subtracted = raw_norm - bkg.background

    # find stars
    daofind = DAOStarFinder(fwhm=dao_fwhm, threshold=dao_threshold * bkg.background_rms_median) 
    sources = daofind(subtracted)
    
    # keep only the NaN-free entries
    has_nan = np.zeros(len(sources), dtype=bool)
    xoff = np.array(sources['xoffset'])
    has_nan |= np.isnan(xoff)
    sources_no_nan = sources[~has_nan]
    

In [ ]:
# build raw image with "normalized" RGB subarrays. Explictly ignore flat field.
raw_norm_1 = imarray * (red_mask * red_norm)
raw_norm_2 = raw_norm_1 + imarray * (green_mask * green_norm)
raw_norm = raw_norm_2 + imarray * (blue_mask * blue_norm)

In [ ]:
# handle saturated pixels
raw_norm = np.where(imarray > 16380, imarray, raw_norm)

In [ ]:
plt.figure(figsize=[9, 5])
print(np.max(raw_norm))
plt.imshow(raw_norm, vmin=0, vmax=2000, cmap='binary')
# plt.imshow(raw_norm, vmax=28000, cmap='gist_stern')
plt.colorbar()

section = raw_norm[750:810,3320:3380]
print("Relative standard deviation of a smooth patch: ", np.std(section) / np.median(section))

In [ ]:
# estimate background
bkg = Background2D(raw_norm, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)

In [ ]:
plt.figure(figsize=[10, 6])
plt.imshow(bkg.background, vmin=0, vmax=1000)
bkg.plot_meshes(outlines=True, color='#1f77b4')
plt.colorbar()

In [ ]:
subtracted = raw_norm - bkg.background

In [ ]:
plt.figure(figsize=[10, 6])
plt.imshow(subtracted, vmin=0, vmax=800, cmap='binary')
plt.colorbar()

In [ ]:
# find star images
daofind = DAOStarFinder(fwhm=dao_fwhm, 
                        threshold=dao_threshold * bkg.background_rms_median)  

# can't be too strict with these. Many images are very non-circular 
# and non-Gaussian due to strong undersampling
#                         sharplo=0.1, sharphi=0.8,
#                         roundlo=-0.7, roundhi=0.7,

sources = daofind(subtracted)  

In [ ]:
for col in sources.colnames:  
    sources[col].info.format = '%.4g'  # for consistent table output
print(sources)  

In [ ]:
# statistics
print("Mean roundness: ", np.average(sources['roundness1']), "stdev: ", np.std(sources['roundness1']))
print("Mean sharpness: ", np.average(sources['sharpness']), "stdev: ", np.std(sources['sharpness']))

In [ ]:
positions = [(x,y) for x,y in zip(sources['xcentroid'], sources['ycentroid'])]
apertures = CircularAperture(positions, r=5.)
plt.figure(figsize=[9, 6])
plt.imshow(subtracted, vmin=-40, vmax=1300, cmap='binary')
plt.colorbar()
ap = apertures.plot(color='red')

## Statistical analysis

These notebook cells (normally not executed) are used to find optimal R and B normalization factors that would minimize roudness (or maximize sharpness).

In [ ]:
# optimization functions
def stats(table):
    mean_roundness = np.average(table['roundness1'])
    mean_sharpness = np.average(table['sharpness'])
    
    print(mean_roundness, mean_shapness)

    return abs(mean_roundness)
#     return 1./ mean_sharpness   # maximum shapness

def objective_function(coeffs):
    
    red_norm = coeffs[0]
    blue_norm = coeffs[1]
    
    raw_norm_1 = imarray * (red_mask * red_norm)
    raw_norm_2 = raw_norm_1 + imarray * (green_mask * 1.0)
    raw_norm = raw_norm_2 + imarray * (blue_mask * blue_norm)
    
    raw_norm = np.where(imarray > 16380, imarray, raw_norm)
    
    bkg = Background2D(raw_norm, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    subtracted = raw_norm - bkg.background
    
    daofind = DAOStarFinder(fwhm=dao_fwhm, threshold=dao_threshold * bkg.background_rms_median)  
    sources = daofind(subtracted)
    
    return stats(sources)  

In [ ]:
# res = minimize(objective_function, (1.3, 1.3), method='Nelder-Mead', tol=1e-2)
# print(res.x, res.fun)

### Conclusion

It appears that optimizing for sharpness or roundness doesn't lead to any significant gain in star image conditioning. Too high sharpness causes star images to become crosses. Too low roundness discards a lot of apparently good star images. 

Assuming that DAOfind is optimizing for a Gaussian profile, we can safely assume that the best approach is to optimize for minimum background scatter and leave it at that.

## Correlate two contiguous images

In [ ]:
# this is the next image after the test image used at the beginning of the notebook
fname2 = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/DSC03780.ARW'
raw2 = rawpy.imread(fname2)
imarray2 = raw2.raw_image_visible.astype(float)

In [ ]:
# normalize
raw_norm_1_2 = imarray2 * (red_mask * red_norm)
raw_norm_2_2 = raw_norm_1_2 + imarray2 * (green_mask * 1.0)
raw_norm2 = raw_norm_2_2 + imarray2 * (blue_mask * blue_norm)

# handle saturated pixels
raw_norm2 = np.where(imarray2 > 16380, imarray2, raw_norm2)

# compute and subtract background
bkg2 = Background2D(raw_norm2, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
subtracted2 = raw_norm2 - bkg2.background

# find stars
daofind = DAOStarFinder(fwhm=dao_fwhm, threshold=dao_threshold * bkg2.background_rms_median) 
sources2 = daofind(subtracted2)

In [ ]:
for col in sources2.colnames:  
    sources2[col].info.format = '%.4g'  # for consistent table output
print(sources2)  

In [ ]:
# plot positions from both first and second images
positions2 = [(x,y) for x,y in zip(sources2['xcentroid'], sources2['ycentroid'])]
apertures2 = CircularAperture(positions2, r=5.)
plt.figure(figsize=[9, 6])
plt.imshow(subtracted2, vmin=-40, vmax=1300, cmap='binary')
plt.colorbar()
ap2 = apertures2.plot(color='red')
ap_ = apertures.plot(color='green') # first image

### Build table with offsets in star positions

First thing to do is to find each pair of stars, one in each table.

For that, we use the centroid positions: for every star in the first table, look for the one star in the second table whose position differs by less than 1.5 pixel (in both X and Y coords).

We also need to store two pointers in the second table. To allow navigation in the list of tables associated to a set of images:

 - the row number of the same star in the reference (first) table.
 - the row number of the same star in the *previous* image in the sequence.
 
The reference image won't have neither of these columns. In the 2nd table in the sequence, these columns will be redundant.

In [ ]:
sources2.add_column(np.nan, name='xoffset')
sources2.add_column(np.nan, name='yoffset')
sources2.add_column(0, name='ref_row')
sources2.add_column(0, name='prev_row') # redundant for the 2nd image in sequence

for row_index in range(len(sources)):
    x = sources[row_index]['xcentroid']
    y = sources[row_index]['ycentroid']
    
    for row2_index in range(len(sources2)):
        x2 = sources2[row2_index]['xcentroid']
        y2 = sources2[row2_index]['ycentroid']
        x_off = x2 - x
        y_off = y2 - y
        if abs(x_off) <= 1.5 and abs(y_off) <= 1.5:
            sources2[row2_index]['xoffset'] = x_off
            sources2[row2_index]['yoffset'] = y_off
            sources2[row2_index]['ref_row'] = row_index
            sources2[row2_index]['prev_row'] = row_index
            
            break # if there is another star that matches the criterion, just ignore it

In [ ]:
for col in sources2.colnames:  
    sources2[col].info.format = '%.5g'  # for consistent table output
print(sources2)  

In [ ]:
# stats
print(np.nanmean(sources2['xoffset']), np.nanstd(sources2['xoffset']))
print(np.nanmean(sources2['yoffset']), np.nanstd(sources2['yoffset']))
print(np.count_nonzero(~np.isnan(sources2['xoffset'])))

In [ ]:
# keep only the NaN-free entries
has_nan = np.zeros(len(sources2), dtype=bool)
xoff = np.array(sources2['xoffset'])
has_nan |= np.isnan(xoff)
sources2_no_nan = sources2[~has_nan]

In [ ]:
for col in sources2_no_nan.colnames:  
    sources2_no_nan[col].info.format = '%.5g'  # for consistent table output
print(sources2_no_nan)

In [ ]:
positions2_n = [(x,y) for x,y in zip(sources2_no_nan['xcentroid'], sources2_no_nan['ycentroid'])]
apertures2_n = CircularAperture(positions2_n, r=5.)
plt.figure(figsize=[9, 6])
plt.imshow(subtracted2, vmin=-40, vmax=1300, cmap='binary')
plt.colorbar()
ap2_n = apertures2_n.plot(color='red')

## Add third image in sequence

The table associated to this image should contain offsets in relation to the first image. But to find star pairs, the code must use the second image. That is, the image immediately before it in the image time sequence.

We may want to add the image name (or path) to the table header.

In [ ]:
# this is the next, third image 
fname3 = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/DSC03781.ARW'
raw3 = rawpy.imread(fname3)
imarray3 = raw3.raw_image_visible.astype(float)

In [ ]:
# normalize
raw_norm_1_3 = imarray3 * (red_mask * red_norm)
raw_norm_2_3 = raw_norm_1_3 + imarray3 * (green_mask * 1.0)
raw_norm3 = raw_norm_2_3 + imarray3 * (blue_mask * blue_norm)

# handle saturated pixels
raw_norm3 = np.where(imarray3 > 16380, imarray3, raw_norm3)

# compute and subtract background
bkg3 = Background2D(raw_norm3, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
subtracted3 = raw_norm3 - bkg3.background

# find stars
daofind = DAOStarFinder(fwhm=dao_fwhm, threshold=dao_threshold * bkg3.background_rms_median)  
sources3 = daofind(subtracted3)

In [ ]:
for col in sources3.colnames:  
    sources3[col].info.format = '%.5g'  # for consistent table output
print(sources3)  

Here we do the more general table operation to correlate the positions just gotten from the 3rd image, with the positions from the second image. But storing offsets in relation to the first (reference) image.

In [ ]:
sources3.add_column(np.nan, name='xoffset')
sources3.add_column(np.nan, name='yoffset')
sources3.add_column(0, name='ref_row')
sources3.add_column(0, name='prev_row')

# loop over rows in 2nd table
for row_index2 in range(len(sources2_no_nan)):
    # get position in 2nd table, and index in 1st table
    x2 = sources2_no_nan[row_index2]['xcentroid']
    y2 = sources2_no_nan[row_index2]['ycentroid']
    ref_row = sources2_no_nan[row_index2]['ref_row']
    
    if ref_row == 0:
        continue

    # loop over rows in 3rd (newest) table
    for row_index3 in range(len(sources3)):
        x3 = sources3[row_index3]['xcentroid']
        y3 = sources3[row_index3]['ycentroid']
        
        # offsets in relation to 2nd table - these are the ones to check for proximity
        x32_off = x3 - x2
        y32_off = y3 - y2
        
        # offsets in relation to reference table
        x_ref = sources[ref_row]['xcentroid']
        y_ref = sources[ref_row]['ycentroid']
        x_off = x3 - x_ref
        y_off = y3 - y_ref

        if abs(x32_off) <= 1.5 and abs(y32_off) <= 1.5:
            sources3[row_index3]['xoffset'] = x_off
            sources3[row_index3]['yoffset'] = y_off
            
            # store pointers to rows in reference and previous tables
            sources3[row_index3]['ref_row'] = ref_row
            sources3[row_index3]['prev_row'] = row_index2
            
            break # if there is another star that matches the criterion, just ignore it

In [ ]:
for col in sources3.colnames:  
    sources3[col].info.format = '%.5g'  # for consistent table output
print(sources3)  

In [ ]:
# keep only the NaN-free entries
has_nan = np.zeros(len(sources3), dtype=bool)
xoff = np.array(sources3['xoffset'])
has_nan |= np.isnan(xoff)
sources3_no_nan = sources3[~has_nan]

In [ ]:
for col in sources3_no_nan.colnames:  
    sources3_no_nan[col].info.format = '%.5g'  # for consistent table output
print(sources3_no_nan)  

In [ ]:
# stats
print(np.nanmean(sources3_no_nan['xoffset']), np.nanstd(sources3_no_nan['xoffset']))
print(np.nanmean(sources3_no_nan['yoffset']), np.nanstd(sources3_no_nan['yoffset']))
print(np.count_nonzero(~np.isnan(sources3_no_nan['xoffset'])))

In [ ]:
# plot positions from 3 images
positions3 = [(x,y) for x,y in zip(sources3['xcentroid'], sources3['ycentroid'])]
apertures3 = CircularAperture(positions3, r=5.)
plt.figure(figsize=[9, 6])
plt.imshow(subtracted3, vmin=-40, vmax=1300, cmap='binary')
plt.colorbar()
ap3 = apertures3.plot(color='blue')
ap2 = apertures2.plot(color='green') 
ap1 = apertures.plot(color='red') 

## Save to file

In [ ]:
sources3_no_nan.write('table.fits', format='fits', overwrite=True)

In [ ]:
test = Table.read('table.fits')

In [ ]:
test